In [3]:
>>> import stanza
>>> stanza.download('en')       # This downloads the English models for the neural pipeline


2020-12-22 15:47:33 INFO: Downloading default packages for language: en (English)...
2020-12-22 15:47:34 INFO: File exists: /Users/johnbellamy/stanza_resources/en/default.zip.
2020-12-22 15:47:37 INFO: Finished downloading models and saved to /Users/johnbellamy/stanza_resources.


In [4]:
>>> nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in 

2020-12-22 15:47:37 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-12-22 15:47:37 INFO: Use device: cpu
2020-12-22 15:47:37 INFO: Loading: tokenize
2020-12-22 15:47:37 INFO: Loading: pos
2020-12-22 15:47:38 INFO: Loading: lemma
2020-12-22 15:47:38 INFO: Loading: depparse
2020-12-22 15:47:39 INFO: Loading: sentiment
2020-12-22 15:47:40 INFO: Loading: ner
2020-12-22 15:47:41 INFO: Done loading processors!


In [990]:
pickle.dump(nlp, open( "nlp.p", "wb" ))

In [963]:
doc = nlp("Where does GM live?")
doc.sentences[0].print_dependencies()

('Where', 4, 'advmod')
('does', 4, 'aux')
('GM', 4, 'nsubj')
('live', 0, 'root')
('?', 4, 'punct')


In [964]:
doc.ents

[{
   "text": "GM",
   "type": "ORG",
   "start_char": 11,
   "end_char": 13
 }]

In [968]:
ner = []

for ent in doc.ents:
    
    ent = ent.to_dict()
    ner.append((ent['text'], ent['type'])) 
    

In [263]:
stop_words = set(stopwords.words('english'))
stop_words = [x for x in stop_words if x != "do" and x != "do?"]

In [265]:
import pickle

 
pickle.dump(stop_words, open( "stops.p", "wb" ) )

In [23]:
import nltk
from py2neo import Graph
import pandas as pd
from pprint import pprint
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import stanza

from nltk import pos_tag
import itertools
from itertools import groupby
import pickle


class NLS:

    def __init__(self, question, nlp):

        self.question = question
        self.__companies__ = pickle.load(open('nls-app/companies.p', 'rb'))
        self.__companies__map__ = pickle.load(open('nls-app/companies_map.p', 'rb'))
        self.__people_companies_map__ = pickle.load(open('nls-app/people_companies.p', 'rb'))
        self.__people_locations_map__ = pickle.load(open('nls-app/people_locations.p', 'rb'))
        self.__nlp__ = nlp
        self.graph = Graph("bolt://localhost:7687", auth = ("neo4j", "test"))
   

    def tokenize(question):
    
        question_tokenized = word_tokenize(question)
        
        stop_words = set(stopwords.words('english'))
        
        filtered_question = [w for w in question_tokenized if not w in stop_words]
        
        filtered_question = []
    
        for w in question_tokenized:
        
            if w not in stop_words:
            
                filtered_question.append(w)
            
        return filtered_question

    def tag(self):
    
        ner, tags = [], []
        
        self.question = self.question.replace("'s","")
        
        if self.question[-1] == 's':
            
            self.question[-1] == ''
            
            
        for pair in itertools.combinations(self.question.split(), 2):
            
            pair = ' '.join(pair)
            
            people1 =  self.__people_companies_map__.get(pair)
            people2 =  self.__people_locations_map__.get(pair)
            
            if people1:
                
                self.question = self.question.replace(pair, people1)
                
            if people2:
        
                self.question = self.question.replace(pair, people2)
                
                
        doc = self.__nlp__(self.question)
                
        for ent in doc.ents:

            ent = ent.to_dict()
            pairs = (ent['text'], ent['type'])
            if pairs not in ner:
                    
                ner.append(pairs)
            
        #displacy.render(doc)
        
        self.filtered_question = NLS.tokenize(self.question)
        
        tags = pos_tag(self.filtered_question)
        
        groups = groupby(tags, key=lambda x: x[1])
        
        names_tagged = [[w for w,_ in words] for tag,words in groups if tag == "NNP"]
        
        names = [" ".join(name) for name in names_tagged if len(name) >= 2]
        
        if len(ner) == 0:
            
            if any([x in self.__companies__ for x in self.filtered_question]):
                
                matches = [x for x in self.__companies__ if x in self.filtered_question]
                
                for m in matches:
                    
                    ner.append(( self.__companies__map__[m], "ORG"))
                
        self.ner = ner
        self.tags = tags 

    def params_builder(self):
    
        params, params_2 = {}, {}
        
        if len(self.ner) == 1:
            
            if (self.ner[0][1] == 'GPE') or (self.ner[0][1] == 'LOC'):
                
                if (self.ner[0][0] == "US") or (self.ner[0][0] == "USA"):
                    
                    country_ = 'United States'
                    
                elif (self.ner[0][0] == "UK"):
                    
                    country_ = 'United Kingdom'
                    
                else:
                    
                    country = self.ner[0][0]
                    
                    params = {}
                    
                    params["country"] = country
                    
            elif (self.ner[0][1] == 'ORG'):
                
                org = self.ner[0][0]
                
                params = {}
                params["org"] = org                    
                
            elif (self.ner[0][1] == 'PERSON'):
                
                person = self.ner[0][0]
                
                params = {}
                
                params["person"] = person
                
        elif len(self.ner) > 1:
            
            name1 = self.ner[0][0]
            
            name2 = self.ner[1][0]
            
            params_2 = {"name1":name1, "name2":name2}
            
        self.params = params 
        self.params_2 = params_2

    def query_picker(self):
    
        label_p = "(p:Person)"
        label_o = "(o:Company)"
        label_l = "(l:NE_Location)"
        works = "-[r:WORKS_FOR]-"
        lives = "-[:LIVES_IN]-"
        
        default ="""match (a:Article)
        return a.Text
        limit 10"""
        
        default_unkown ="""match (a:Article)
        where 
        return a.Text
        limit 10"""

        query1 = '''
        match {} {} {}
        where p.value = $person
        return p.value as Name,o.value as WORKS_FOR
        '''.format(label_p,works,label_o)

        query1_1 = '''
        match {} {} {}
        where o.value = $org
        return o.value as Organization, collect(distinct p.value) as Person, r.value as Position
        '''.format(label_p,works,label_o)

        query2 = '''
        match (p:Person {value:$person})-[r:LIVES_IN]-(l:Location)
        return p.value as Person, l.value as STAYS_AT
        '''

        query3 = '''
        MATCH (p1:NER_Person:Tag{ value: $name1 }),(p2:NER_Person:Tag{ value: $name2 }), p = shortestPath((p1)-[*..15]-(p2))
        RETURN p1.value as Person1, p2.value as Person2, p as Relation
        '''

        query4 = '''
        MATCH (p:NER_Person)-[w:LIVES_IN]->(o:NER_Location)
        RETURN p.value as Person ,o.value as Lives_in
        '''.format(label_p,lives,label_o)

        query5 = '''
        MATCH (p:NER_Person)-[:LIVES_IN]->(l:NER_Location), (p)-[w:WORKS_AT]-(o:NER_Organization)
        RETURN p.value as Person, l.value as Lives_in, o.value as Works_at, w.AS as Position
        '''

        query6 = '''
        MATCH (s:Sentence)-[st:SENTENCE_TAG_OCCURRENCE]->(n:TagOccurrence), (s)-[h:HAS_TAG]-(p:NER_Person), (s)-[h]-(o:NER_Organization)
        where n.value IN ["said","says","think","thinks"] AND (p.value in $names OR o.value in $org)
        return s.text as Sentence, p.value as Person
        '''

        query7 = '''
        match (a:Article)-[:HAS_ANNOTATED_TEXT]-(at:AnnotatedText)-[:CONTAINS_SENTENCE]-(s:Sentence)-[:SENTENCE_TAG_OCCURRENCE]-(t:TagOccurrence)-[:TAG_OCCURRENCE_TAG]-(o:NER_Organization)
        where o.value = $org
        return distinct a.Text

        '''

        query8 = '''
        match (a:Article)-[:HAS_ANNOTATED_TEXT]-(at:AnnotatedText)-[:CONTAINS_SENTENCE]-(s:Sentence)-[:SENTENCE_TAG_OCCURRENCE]-(t:TagOccurrence)-[:TAG_OCCURRENCE_TAG]-(p:NER_Person)
        where p.value = $person
        return distinct a.Text

        '''
        
        query9 = '''
        match (p:Person {value: $person})-[:HAS_TITLE]-(t:Title)
        return distinct p.value as Person, t.value as HAS_TITLE
        '''
        
        tags_list = [x[0] for x in self.tags]
        pos_tag_list = [x[1] for x in self.tags]
        value_list = []

        if not self.tags and not self.params:
            
            self.response = [self.graph.run(default).data(), "article", "null"]
            
        if self.tags and self.params and not all(e == 'NNP' for e in pos_tag_list) and not all(e == 'NN' for e in pos_tag_list):
            
            for _, value in self.params.items():
                
                for v in value.split():
                    
                    if v not in value_list:
                        
                        value_list.append(v)

            if all(value_list for a in tags_list):

                non_param_tags = [x for x in tags_list if x not in value_list]

                for word in non_param_tags:


                    if word in ['work','do']: # look verbs and see if there is a work verb 

                        self.response = [self.graph.run(query1, self.params).data(), "rdf-triple"]

                    elif word in ['works','at']:

                        self.response = [self.graph.run(query1_1, self.params).data(), "rdf-triple"]
                    
                    elif word in ['title','job','role','position', 'job?']:

                        self.response = [self.graph.run(query9, self.params).data(), "rdf-triple"]

                    elif word in ['live','reside','stay', 'live?']:
                        
                        print("live")
                        
                        try:
                            
                            if len(self.params.get('person').split()) == 1:
                            
                                words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP" or x[1] == "NN"] 
                
                                params = {"person":" ".join(words)}
                    
                            else:
                            
                                params = self.params
                            
                        except:
                            
                            if len(self.params.get('org').split()) == 1:
                            
                                words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP" or x[1] == "NN"] 
                
                                params = {"person":" ".join(words)}
                         
                    
                            else:
                            
                                params = self.params
                                
                     

                        self.response = [self.graph.run(query2, params).data(), "rdf-triple"]

                    elif word in ['related','relation']:

                        self.response = self.graph.run(query3, self.params_2).data()


                    elif word in ['everyone']:

                        self.response = self.graph.run(query5).data()

                    elif word in ['think', 'says', 'say']:

                        self.response = self.graph.run(query6, self.params).data() 
                        
                    elif word in ['article','articles', 'Article','Articles']:
                        

                        if self.params.get('org'):
                            

                            self.response = [self.graph.run(query7, self.params).data(), "article", self.params['org']]

                        else:
                            print("else")

                            words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP"] 

                            params = {"person":" ".join(words)}
                            self.response = [self.graph.run(query8, params).data(), "article", params['person']]
                            
        
        else:
            
            print("else")
            
            if self.params.get('person'):
                print('person')
                
                words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP" or x[1] == "NN"] 
                print(words)
                
                params = {"person":" ".join(words)}
                        
                self.response = [self.graph.run(query8, params).data(), "article", params['person']]
                
            if self.params.get('org'):
                print("two")  
                self.response = [self.graph.run(query7, self.params).data(), "article", self.params['org']]
                
            else:
                
                term = self.tags[0][0][0].upper() + self.tags[0][0][1:]
                response1 = self.graph.run(query7, {'org':term}).data()
                response2 = self.graph.run(query8, {'person':term}).data()
                
                if response1:
                    
                    self.response = [response1, "article", term]
                    
                elif response2:
                    
                    self.response = [response2, "article", term]

In [24]:
nls.__companies__map__.get('fiat')

'Fiat'

In [28]:
nls = NLS("what does xie xuren do?", nlp)

In [29]:
[person for person in nls.__people_companies_map__.keys() if person in nls.__people_locations_map__.keys()]

['emmanuel gaillard']

In [30]:
nls.tag()
nls.params_builder()
nls.query_picker()

In [33]:
nls.response()

AttributeError: 'NLS' object has no attribute 'response'

In [32]:
try:
    
    if nls.response[1] == "article":
    
        articles = []
    
        for article in nls.response[0]:
           
            articles.append(article)
        
        results = {"responseType": "article", "entity": nls.response[2], "articles":articles}
    
    if nls.response[1] == "rdf-triple":

        try:

            results = {'responseType': "rdf-triple",
                   'subject': nls.response[0][0][list(dict(nls.response[0][0]).keys())[0]],
                   'object': nls.response[0][0][list(dict(nls.response[0][0]).keys())[1]],

                   'predicate': list(dict(nls.response[0][0]).keys())[1]}

            if type(results.get('object')) == list:

                results['object'] = " & ".join(results.get('object'))

        except IndexError:

            results =  {'responseType': "rdf-triple",
                   'subject': "",
                   'object': "",
                   'predicate': ""}
            
except:
    
    results =  {'responseType': "rdf-triple",
                   'subject': "",
                   'object': "",
                   'predicate': ""}


    

In [290]:
results

{'responseType': 'rdf-triple',
 'subject': 'Xie Xuren',
 'object': 'head',
 'predicate': 'HAS_TITLE'}

In [286]:
nls.filtered_question

['Louisa', 'Lim', 'live']

In [1546]:
nls.response(list(dict(nls.response[0][0]).keys())[0])

TypeError: 'list' object is not callable

In [ ]:
nls.response

In [596]:
"hello"[0].isupper()

False

In [620]:
"hello"[0][0].upper() + "hello"[1:] 

'Hello'

In [807]:
ner, tags = [], []
question = "where does Nestor Kirchner live?"        
nlp = en_core_web_sm.load()

doc = nlp(question)

ner = [(X.text, X.label_) for X in doc.ents]

#displacy.render(doc)

tags = pos_tag(words)

groups = groupby(tags, key=lambda x: x[1])

names_tagged = [[w for w,_ in words] for tag,words in groups if tag=="NNP"]

names = [" ".join(name) for name in names_tagged if len(name)>=2]


In [808]:
names_tagged

[[' '], [' ', 'N'], [' ', 'K', 'i'], [' ']]

In [810]:
ner

[('Nestor Kirchner', 'PERSON')]

In [811]:
words

'where does Nestor Kirchner live?'

In [817]:
nls.params


{'person': 'Kirchner'}

In [919]:
!ls

API Test.ipynb
CXFBISOLR.pem
Entity Linking.ipynb
Knowledge Graphs.pdf
Load Data.ipynb
NLS Notes.docx
Neo4j-NLP
Notes about NLS.docx
Query Test.ipynb
UIMA Solr Notes.docx
USPTO IAISS Trademark Image Search & ID RFI - Octo - 11022020.docx
__pycache__
article_response.json
bbc
build
cities1000.txt
cypher_queries.cypher
dwh_evidence.csv
dwh_evidence.txt
dwh_evidence.zip
evidence.csv
evidence_1130.json
evidence_1207.json
jobs_1120.json
jobs_41.json
monster-jobs.csv
nginx.conf
nls-app
nls-app.zip
nls.py
nltk_contrib
queries.json
rdf.py
schema.xml
semantic-knowledge-graph
solr notes.docx
statistical-phrase-identifier


In [ ]:
import nltk
from py2neo import Graph
import pandas as pd
from pprint import pprint
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import spacy
from spacy import displacy
import en_core_web_sm
from nltk import pos_tag
from itertools import groupby
import pickle


class NLS:

    def __init__(self, question):

        self.question = question
        self.companies = pickle.load(open('nls-app/companies.p', 'rb'))
        self.companies_map = pickle.load(open('nls-app/companies_map.p', 'rb'))
        self.people_companies_map = pickle.load(open('nls-app/people_companies.p', 'rb'))
        self.people_locations_map = pickle.load(open('nls-app/people_locations.p', 'rb'))


    def connect():
        global graph
        graph = Graph("bolt://ec2-18-191-135-163.us-east-2.compute.amazonaws.com:7687", auth = ("neo4j", "test"))
        tx = graph.begin()
    
    def tokenize(self):
    
        question_tokenized = word_tokenize(self.question)
        
        stop_words = set(stopwords.words('english'))
        
        filtered_question = [w for w in question_tokenized if not w in stop_words]
        
        filtered_question = []
    
        for w in question_tokenized:
        
            if w not in stop_words:
            
                filtered_question.append(w)
            
        self.filtered_question = filtered_question

    def tag(self):
    
        ner, tags = [], []
        
        nlp = en_core_web_sm.load()
        
        doc = nlp(self.question)
        
        ner = [(X.text, X.label_) for X in doc.ents]
            
        #displacy.render(doc)
        
        tags = pos_tag(self.filtered_question)
        
        groups = groupby(tags, key=lambda x: x[1])
        
        names_tagged = [[w for w,_ in words] for tag,words in groups if tag=="NNP"]
        
        names = [" ".join(name) for name in names_tagged if len(name)>=2]
        
        if len(ner) == 0:
            
            if any([x in self.companies for x in self.filtered_question]):
                
                matches = [x for x in self.companies if x in self.filtered_question]
                
                for m in matches:
                    
                    ner.append((m, "ORG"))
                
        self.ner = ner
        self.tags = tags 

    def params_builder(self):
    
        params, params_2 = {}, {}
        
        if len(self.ner) == 1:
            
            if (self.ner[0][1] == 'GPE') or (self.ner[0][1] == 'LOC'):
                
                if (self.ner[0][0] == "US") or (self.ner[0][0] == "USA"):
                    
                    country_ = 'United States'
                    
                elif (self.ner[0][0] == "UK"):
                    
                    country_ = 'United Kingdom'
                    
                else:
                    
                    country = self.ner[0][0]
                    
                    params = {}
                    
                    params["country"] = country
                    
            elif (self.ner[0][1] == 'ORG'):
                
                org = self.ner[0][0]
                
                params = {}
                
                try:
                    params["org"] = companies_map[org]
                
                except:
                
                    if org[0].islower():
                    
                        params["org"] = org[0].upper() + org[1:]
                    
                    else:
                    
                        params["org"] = org                    
                
            elif (self.ner[0][1] == 'PERSON'):
                
                person = self.ner[0][0]
                
                params = {}
                
                params["person"] = person
                
        elif len(self.ner) > 1:
            
            name1 = self.ner[0][0]
            
            name2 = self.ner[1][0]
            
            params_2 = {"name1":name1, "name2":name2}
            
        self.params = params 
        self.params_2 = params_2

    def query_picker(self):
    
        label_p = "(p:Person)"
        label_o = "(o:Company)"
        label_l = "(l:NE_Location)"
        works = "-[r:WORKS_FOR]-"
        lives = "-[:LIVES_IN]-"
        
        default ="""match (a:Article)
        return a.Text
        limit 10"""
        
        default_unkown ="""match (a:Article)
        where 
        return a.Text
        limit 10"""

        query1 = '''
        match {} {} {}
        where p.value = $person
        return p.value as Name,o.value as WORKS_FOR
        '''.format(label_p,works,label_o)

        query1_1 = '''
        match {} {} {}
        where o.value = $org
        return o.value as Organization, collect(distinct p.value) as Person, r.value as Position
        '''.format(label_p,works,label_o)

        query2 = '''
        match (p:Person {value:$person})-[r:LIVES_IN]-(l:Location)
        return p.value as Person, l.value as STAYS_AT
        '''

        query3 = '''
        MATCH (p1:NER_Person:Tag{ value: $name1 }),(p2:NER_Person:Tag{ value: $name2 }), p = shortestPath((p1)-[*..15]-(p2))
        RETURN p1.value as Person1, p2.value as Person2, p as Relation
        '''

        query4 = '''
        MATCH (p:NER_Person)-[w:LIVES_IN]->(o:NER_Location)
        RETURN p.value as Person ,o.value as Lives_in
        '''.format(label_p,lives,label_o)

        query5 = '''
        MATCH (p:NER_Person)-[:LIVES_IN]->(l:NER_Location), (p)-[w:WORKS_AT]-(o:NER_Organization)
        RETURN p.value as Person, l.value as Lives_in, o.value as Works_at, w.AS as Position
        '''

        query6 = '''
        MATCH (s:Sentence)-[st:SENTENCE_TAG_OCCURRENCE]->(n:TagOccurrence), (s)-[h:HAS_TAG]-(p:NER_Person), (s)-[h]-(o:NER_Organization)
        where n.value IN ["said","says","think","thinks"] AND (p.value in $names OR o.value in $org)
        return s.text as Sentence, p.value as Person
        '''

        query7 = '''
        match (a:Article)-[:HAS_ANNOTATED_TEXT]-(at:AnnotatedText)-[:CONTAINS_SENTENCE]-(s:Sentence)-[:SENTENCE_TAG_OCCURRENCE]-(t:TagOccurrence)-[:TAG_OCCURRENCE_TAG]-(o:NER_Organization)
        where o.value = $org
        return distinct a.Text

        '''

        query8 = '''
        match (a:Article)-[:HAS_ANNOTATED_TEXT]-(at:AnnotatedText)-[:CONTAINS_SENTENCE]-(s:Sentence)-[:SENTENCE_TAG_OCCURRENCE]-(t:TagOccurrence)-[:TAG_OCCURRENCE_TAG]-(p:NER_Person)
        where p.value = $person
        return distinct a.Text

        '''
        
        tags_list = [x[0] for x in self.tags]
        pos_tag_list = [x[1] for x in self.tags]
        value_list = []

        if not self.tags and not self.params:
            
            self.response = [graph.run(default).data(), "article", "null"]
            
        if self.tags and self.params and not all(e == 'NNP' for e in pos_tag_list) and not all(e == 'NN' for e in pos_tag_list):
            
            for _, value in self.params.items():
                
                for v in value.split():
                    
                    if v not in value_list:
                        
                        value_list.append(v)

            if all(value_list for a in tags_list):

                non_param_tags = [x for x in tags_list if x not in value_list]

                for word in non_param_tags:


                    if word in ['work','do']:

                        self.response = [graph.run(query1, self.params).data(), "rdf-triple"]

                    elif word in ['works','at']:

                        self.response = [graph.run(query1_1, self.params).data(), "rdf-triple"]

                    elif word in ['live','reside','stay', 'live?']:
                        
                        print("live")
                        
                        try:
                            
                            if len(self.params.get('person').split()) == 1:
                            
                                words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP" or x[1] == "NN"] 
                
                                params = {"person":" ".join(words)}
                    
                            else:
                            
                                params = self.params
                            
                        except:
                            
                            if len(self.params.get('org').split()) == 1:
                            
                                words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP" or x[1] == "NN"] 
                
                                params = {"person":" ".join(words)}
                         
                    
                            else:
                            
                                params = self.params
                                
                        print(" ".join(words))

                        self.response = [graph.run(query2, params).data(), "rdf-triple"]

                    elif word in ['related','relation']:

                        self.response = graph.run(query3, self.params_2).data()


                    elif word in ['everyone']:

                        self.response = graph.run(query5).data()

                    elif word in ['think', 'says', 'say']:

                        self.response = graph.run(query6, self.params).data() 
                        
                    elif word in ['article','articles', 'Article','Articles']:
                        

                        if self.params.get('org'):
                            

                            self.response = [graph.run(query7, self.params).data(), "article"]

                        else:
                            print("else")

                            words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP"] 

                            params = {"person":" ".join(words)}
                            self.response = [graph.run(query8, params).data(), "article", params['person']]
                            
        
        else:
            
            print("else")
            
            if self.params.get('person'):
                print('person')
                
                words = [x[0][0].upper() + x[0][1:] for x in self.tags if x[1] == "NNP" or x[1] == "NN"] 
                print(words)
                
                params = {"person":" ".join(words)}
                        
                self.response = [graph.run(query8, params).data(), "article", params['person']]
                
            if self.params.get('org'):
                print("two")  
                self.response = [graph.run(query7, self.params).data(), "article", self.params['org']]
                
            else:
                
                term = self.tags[0][0][0].upper() + self.tags[0][0][1:]
                response1 = graph.run(query7, {'org':term}).data()
                response2 = graph.run(query8, {'person':term}).data()
                
                if response1:
                    
                    self.response = [response1, "article", term]
                    
                elif response2:
                    
                    
                    self.response = [response2, "article", term]
                    
    
                
                
                
            
            
     